Grab the modules - note also the two files you need to create, credentials.py to hold your twitter API tokens and settings.py for constants.

We're using tweepy for Twitter API access http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweep

In [2]:
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob


This is Main function.
Extracting streaming data from Twitter, pre-processing, and loading into MySQL

Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html


We are using textblog for sentiment analyses, which is a cool library. See https://www.analyticsvidhya.com/blog/2018/02/natural-language-processing-for-beginners-using-textblob/

In [3]:
# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    # Extract info from tweets
    def on_status(self, status):
        if status.retweeted:
        # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        # We remove emojis (see code later on)
        text = deEmojify(status.text)    
        # We use textblog to determine sentiment
        sentiment = TextBlob(text).sentiment
        # Polarity is float which lies in the range of [-1,1]
        # where 1 means positive statement and -1 means a negative statement.
        polarity = sentiment.polarity
        # Subjective sentences generally refer to personal opinion, emotion or judgment whereas
        # objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    # Twitter API has rate limits, so we stop scraping data if it exceeds the threshold.
    def on_error(self, status_code):
        if status_code == 420:
            # return False to disconnect the stream
            return False

Here we have the functions to clean up tweet text -- removing links, special characters, and emojis.

In [4]:
# Use simple regex statements to clean tweet text by removing links and special characters
def clean_tweet(self, tweet): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
# Get rid of emojis by stripping all non-ASCII characters
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

Here's the function to connect to MySQL. For some reason, this was a HUGE pain in the ass on my mac. I installed MySQL using the standard packages, but it absolutely would NOT accept the root user password that was created during installation. I have no idea WTF was going on there. I reinstalled three times with no luck.

In the end I just used MAMP, which is a web server/MySQL package I use for web developement. To use it I had to add the unix socket variable below. This took an incredible amount of time to figure out.

In [5]:
mydb = mysql.connector.connect(
    host="localhost",
    port="3306",
    unix_socket="/Applications/MAMP/tmp/mysql/mysql.sock",
    user="chris",
    passwd="twitter",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    # Check if this table exits. If not, then create a new one.
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

Here we define the Twitter authentication using the credentials.py we imported at the beginning

In [6]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

And we initiate the API

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @dominiquetaegon: I just came across this video on Facebook. 

This man is basically crying tears of happiness because the govt’s ‘bounc…
Long: None, Lati: None
RT @NuneatonGriff: Ady will be going live on Facebook  @6pm to reveal the winner of the blackout. Video will be posted straight away. https…
Long: None, Lati: None
RT @Jordan_Sather_: Did I mention that Facebook will no longer let me run promotions on my 50,000 follower Destroying the Illusion page bec…
Long: None, Lati: None
RT @Realoilsheikh: You can join the Tafseer of Shaykh @DrIsaPantami live from Annur Masjid, Wuse Abuja.

https://t.co/gz8SkH9P9J
Long: None, Lati: None
@donnaet5 @Avenger2Toxic 🙄 go figure. They stopped trying to debunk the truth and are now out right silencing it. @Facebook @Twitter
Long: None, Lati: None
love Facebook memes
also this mentality is basically why I was popular in highschool https://t.co/ndTGcAqjXO
Long: None, Lati: None
RT @MalwareTechBlog: Did anyone really really want to be popular in 

The left doesn't want us to go back to work, they want us to depend on them, lets us lose our jobs, our homes, cars… https://t.co/BOdHbM546y
Long: None, Lati: None
RT @dns_official__: Meanwhile on Facebook...... https://t.co/F2nafyfTOV
Long: None, Lati: None
RT @aruguer: When Current CM does facebook live people invites him to play PUBG and when Ex CM does Facebook Live people expects him to del…
Long: None, Lati: None
RT @3_K2H: LMAFOO Facebook niggas have no heart https://t.co/yXoJGBjiTq
Long: None, Lati: None
#NEW The 360 edition of the Montreal Pride Festival will be held August 10 to 16th.

https://t.co/1p9O4uqxw9 

(pho… https://t.co/Z97tN0o7Kf
Long: None, Lati: None
RT @ReadingIsOurPas: You're invited and most welcome to share ANYTHING book related,  ALL genres, on our page,  ANYTIME.  

Read a great bo…
Long: None, Lati: None
RT @LeobaneCosplay: "My shirt ? Oh yeah, I removed it earlier. But honestly, I'm too hot to get cold anyway." 🔥
--
Pic: pmforeverarts (IG)…
Long: None, La

RT @VioletTamaskan: There's no greater and heart warming redemption arc than the Tails from "Sonic" is NOT gay! facebook page https://t.co/…
Long: None, Lati: None
RT @Lawren_Leo: Celebrate the release of Horse Magick with me, LIVE on Facebook!
https://t.co/KLUCd08VBb
#horsemagick #LiveEvent https://t.…
Long: None, Lati: None
@IPR_Odisha @STAOdisha https://t.co/ZILJ3DlUiO
Please support to migrant worker
Long: None, Lati: None
@RealCandaceO @CNN @ToddMcMurtry @Facebook You go girl!
Long: None, Lati: None
Harry Potter's Daniel Radcliffe baffled his co-stars are 'old enough to have kids' https://t.co/b4J79byVHE https://t.co/WeDHlO1I9u
Long: None, Lati: None
RT @ReadingIsOurPas: You're invited and most welcome to share ANYTHING book related,  ALL genres, on our page,  ANYTIME.  

Read a great bo…
Long: None, Lati: None
RT @gatewaypundit: SHOCKING DEVELOPMENT! Singapore Woman Linked to Bill Gates, Soros and China Is Flagging and Removing ALL VIDEO CONTENT b…
Long: None, Lati: None
RT @LCMS

RT @WORLDMUSICAWARD: #EXO’s #Baekhyun will premiere his new song #Candy on May 26 and shares more adorable pics and a sweet mood teaser for…
Long: None, Lati: None
#SaveKoreanDogs our #SuccessStories 

Sharing from Cynthia White Meyer

"Anna and Loki,
LOVE STORY"

#SouthKorea… https://t.co/pNtt0Npw4p
Long: None, Lati: None
RT @AlfredStreetBC: #ASBC Virtual Bible Study Today @ 7pm
Join us for another session of "Can I Push It?" w/ @PastorHJW
Come prepared to pr…
Long: None, Lati: None
RT @bruhziIl: Facebook wildin 🤣🤣 https://t.co/TUPxHc9kml
Long: None, Lati: None
RT @Gabe1Herrera: Get your gear ‼️‼️‼️
Long: None, Lati: None
RT @WORLDMUSICAWARD: #EXO’s #Baekhyun will premiere his new song #Candy on May 26 and shares more adorable pics and a sweet mood teaser for…
Long: None, Lati: None
RT @but_seekan: @DgpKarnataka @TelanganaDGP @DGP_FIRE @TheKeralaPolice @suhas06078772 @factcheckksp 
Sir, this man is provoking the publicl…
Long: None, Lati: None
RT @WORLDMUSICAWARD: #EXO’s #Baekhyun wil

Oh man, so giddy!
Long: None, Lati: None
RT @EasterSealsNL: Still looking for a reason to buy a Cabin Lottery Ticket? 🌲

Head over to our Facebook page to check out the interior ph…
Long: None, Lati: None
RT @katyperry: Tune in TONIGHT for the #AmericanIdol grand finale AND my first television performance of #Daisies! 🌼 It all starts at 8/7c…
Long: None, Lati: None
RT @bruhziIl: Facebook wildin 🤣🤣 https://t.co/TUPxHc9kml
Long: None, Lati: None
RT @prageru: BREAKING:

@Facebook has reduced our page reach and implemented restrictions for "repeated sharing of false news."

Facebook i…
Long: None, Lati: None
RT @WORLDMUSICAWARD: #GOT7’s #JB Oozes Charisma In Latest Cover Shoot For The June Issue Of #NylonKorea, And Opens Up In Cover Story About…
Long: None, Lati: None
RT @WORLDMUSICAWARD: #EXO’s #Baekhyun will premiere his new song #Candy on May 26 and shares more adorable pics and a sweet mood teaser for…
Long: None, Lati: None
RT @HarvardChanSPH: Forum event is starting now at https://t.

@Odunadekolade You want to download video?

Click on link below 👇 to download videos from twitter, Instagram and Fa… https://t.co/DSPukkV6w8
Long: None, Lati: None
I fw my Facebook avatar https://t.co/tKJVTLG3MQ
Long: None, Lati: None
